In [1]:
import pandas as pd
import os
import csv
import pickle
from nltk import word_tokenize, pos_tag
from nltk import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
import nltk
nltk.download('wordnet')
import warnings
warnings.simplefilter("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\RYK\AppData\Roaming\nltk_data...


In [2]:
def find_the_way():
    path = './models/'
    models_files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.npy' not in file:
                models_files.append(os.path.join(r, file))  
    return models_files
models_files=find_the_way()
print(models_files)

['./models/model_1000_10_cbow', './models/model_1000_10_skipgram', './models/model_1000_4_cbow', './models/model_1000_4_skipgram', './models/model_300_10_cbow', './models/model_300_10_skipgram', './models/model_300_4_cbow', './models/model_300_4_skipgram', './models/model_600_10_cbow', './models/model_600_10_skipgram', './models/model_600_4_cbow', './models/model_600_4_skipgram']


In [3]:
with open('animal_list_snowball.pickle', 'rb') as handle:
    anim_dict = pickle.load(handle)
anim_dict

{'yerdomuzu': 'yerdo',
 'antilop': 'antilop',
 'bizon': 'bizo',
 'deve': 'dev',
 'bukalemun': 'bukale',
 'çita': 'çita',
 'şempanze': 'şempanze',
 'kobra': 'kobra',
 'afrikageyiği': 'afrikageyik',
 'fil': 'fil',
 'ceylan': 'ceyla',
 'zürafa': 'züraf',
 'öküzbaşlıantilop': 'öküzbaşlıantilop',
 'goril': 'goril',
 'suaygırı': 'suaygır',
 'sırtlan': 'sırtla',
 'impala': 'impal',
 'çakal': 'çakal',
 'lemur': 'lemur',
 'leopar': 'leopar',
 'aslan': 'asla',
 'denizineği': 'denizinek',
 'firavunfaresi': 'firavunfares',
 'maymun': 'maym',
 'devekuşu': 'devekuş',
 'panter': 'panter',
 'gergedan': 'gergeda',
 'kaplan': 'kapla',
 'afrikaantilobu': 'afrikaantilop',
 'yabandomuzu': 'yabando',
 'zebra': 'zebra',
 'pars': 'pars',
 'çıta': 'çı',
 'mirket': 'mirket',
 'şebek': 'şebek',
 'hipopotam': 'hipopota',
 'arslan': 'arsla',
 'gepard': 'gepardı',
 'emu': 'emu',
 'kanguru': 'kangur',
 'kivi': 'kiv',
 'keselisıçan': 'keselisıça',
 'ornitorenk': 'ornitorenk',
 'tazmanyacanavarı': 'tazmanyacanavar',
 

In [4]:
#This code creates three clusters file for 0.25,0.50,0.75 percentages as a w2v_list_0.25.txt, w2v_list_0.5.txt, w2v_list_0.75.txt
#Also creates sorted word2vec cosine similarity results for all animal pairs as a w2v_scores.csv
#And creates scores file in a txt to check manually as a scr_list.txt. This file includes similarity scores for every person.

#reading animal lists(cvf sequences)
animals = pd.read_csv("animal.csv") #reading a file
data=animals["PREPROCESSED TEXT"].values
ids=animals["ID"].values
#loading models that found in previous step(we have 4 models in here)
for m in models_files:
    model = KeyedVectors.load_word2vec_format(m)
    modelname="./word2vec_cluster_outputs/"+str(m)[9:]
    
    #calculating similarity scores and assigning w2v_scores list
    w2v_scores=[]
    for i in data:
        j=i.replace(" ",",")
        patient=list(j.split(","))
        while True:
            try:
                patient.remove('')
            except:
                break
        for j in range(0,len(patient)-1):    
            #word2vec cosine similarity
            w1=patient[j]
            w2=patient[j+1]
            try:
                w2v=model.similarity(anim_dict[w1],anim_dict[w2])
                if w1==w2:
                    continue
                else:
                    splitting=(str(w2v)+"@"+str(w1)+"@" +str(w2))
                    if splitting not in w2v_scores:
                        w2v_scores.append(str(w2v)+"@"+str(w1)+"@" +str(w2))
            except:
                print(w1,w2)         


    #sorting similarity scores 
    w2v_scores.sort()

    #writing sorted similarity scores to CSV file
    score_file_name=modelname+"_w2v_scores.csv"
    with open(score_file_name, "w", newline="",encoding="utf-8") as f:
        wrt = csv.writer(f)
        wrt.writerow(["similarity_scores","animal_1", "animal_2"])
        for i in range(0,len(w2v_scores)):
            splitting=w2v_scores[i].split("@")
            wrt.writerow([splitting[0],splitting[1],splitting[2]])

    #calculating threshold values for 0.25,0.50,0.75 percentage
    threshold_list=[0.25,0.50,0.75]  
    score_flag=True
    count=0
    for t in threshold_list:
        w2v_switch=[]
        ########## writing w2v_threshold values in a screen
        w2v_threshold=w2v_scores[int(len(w2v_scores)*t)].split("@")
        print(t,modelname," Threshold for Word2vec is:",w2v_threshold[0])
        w2v_threshold=float (w2v_threshold[0])

        ########creation of word2nvec clusters
        big_animal_list2=[]
        big_animal_list2_score=[]
        for i in data:
            j=i.replace(" ",",")
            patient=list(j.split(","))
            while True:
                try:
                    patient.remove('')
                except:
                    break
            temp=[]
            temp_score=[]
            flag=0
            animal_list=[]
            for j in range(0,len(patient)-1):    
                #calculating word2vec cosine similarity for creating clusters
                w1=patient[j]
                w2=patient[j+1]
                try:
                    w2v=model.similarity(anim_dict[w1],anim_dict[w2])
                    temp_score.append(round(w2v,3))

                    if w1==w2:
                        continue
                    else:
                        if w2v>=w2v_threshold:
                            if flag==1:
                                temp+=[w2]                
                            else:
                                temp+=[w1,w2]
                                flag=1
                        else:
                            if temp!=[]:
                                animal_list+=[temp]
                                temp=[]
                                flag=0
                            else:
                                animal_list+=[w1]
                except:
                    print(w1,w2)
        #creating word2vec clusters and assign a list (temp list)
            if temp!=[]:
                animal_list+=[temp]
            else:
                animal_list+=[w2]

            big_animal_list2.append(animal_list)
            big_animal_list2_score.append(temp_score)
        #################### make alone string names(singleton) as a group
        w2v_list=[]
        for i in big_animal_list2:
            p=[]
            for ii in i:
                if ii.__class__ == str:
                    p.append([ii])
                else:
                    p.append(ii)
            w2v_list.append( p ) 
        #calculating switch numbers.(the number of created clusters-1 equals switch counts.)
        for i in w2v_list:
            w2v_switch.append(len(i)-1)
        #assigning switch numbers to pnada dataframe
        if score_flag:
            df_switch = pd.DataFrame(w2v_switch, columns = [t]) 
        else:
            df_switch.insert(count,str(t), w2v_switch)


        w2v_list_name=modelname+"_w2v_list_"+str(t)+".txt"
        #writing w2v_list to CSV file. Can be seen created all clusters between bracelets[]
        ths = open(w2v_list_name, "w", encoding="utf-8")
        #ths.write("ID@CLUSTER\n")
        for sayac,i in enumerate(w2v_list):
            #temp=str(ids[sayac])+"@"+str(i)+"\n"
            temp=str(i)+"\n"
            ths.write(temp)
        ths.close()
        count=count+1
        #writing scores in a txt file to check clusters manually-optional
        if score_flag:
            scr_list_name=modelname+"_scr_list.txt"
            ths = open(scr_list_name, "w", encoding="utf-8")
            #ths.write("ID@CLUSTER\n")
            for sayac,i in  enumerate(big_animal_list2_score):
                #temp=str(ids[sayac])+"@"+str(i)+"\n"
                temp=str(i)+"\n"
                ths.write(temp)
            ths.close()
            score_flag=False
    switchcounts_name=modelname+"_w2v_switch.csv"
    #df_switch["ID"]=ids
    #writing switch numbers(threshold 0.25,0.50 and 0.75 by percentage) in a csv file.the file name is modelname_w2v_switch.csv
    df_switch.to_csv(switchcounts_name,index=False)

0.25 model_1000_10_cbow  Threshold for Word2vec is: 0.16312341
0.5 model_1000_10_cbow  Threshold for Word2vec is: 0.29870936
0.75 model_1000_10_cbow  Threshold for Word2vec is: 0.4428481
0.25 model_1000_10_skipgram  Threshold for Word2vec is: 0.13611728
0.5 model_1000_10_skipgram  Threshold for Word2vec is: 0.1990204
0.75 model_1000_10_skipgram  Threshold for Word2vec is: 0.2678917
0.25 model_1000_4_cbow  Threshold for Word2vec is: 0.17810473
0.5 model_1000_4_cbow  Threshold for Word2vec is: 0.32770053
0.75 model_1000_4_cbow  Threshold for Word2vec is: 0.4660498
0.25 model_1000_4_skipgram  Threshold for Word2vec is: 0.15042028
0.5 model_1000_4_skipgram  Threshold for Word2vec is: 0.22613452
0.75 model_1000_4_skipgram  Threshold for Word2vec is: 0.3018213
0.25 model_300_10_cbow  Threshold for Word2vec is: 0.19835623
0.5 model_300_10_cbow  Threshold for Word2vec is: 0.35215443
0.75 model_300_10_cbow  Threshold for Word2vec is: 0.48747554
0.25 model_300_10_skipgram  Threshold for Word2vec